# Ben Needs a Friend - In-Context Learning

This is part of the "Ben Needs a Friend" tutorial. See all the notebooks and materials [here](https://github.com/bpben/ben_friend). Follow setup instructions there to use this notebook.

In this notebook, I provide a brief intro on how we'll be setting up and interacting with LLMs.

## Table of Contents
1. [Pre-trained models](#pre-trained-models)
    - [Temperature](#temperature)
2. [Instruction tuning](#instruction-tuning)
3. [System prompts](#system-prompts)


In [2]:
from llamabot import SimpleBot, ChatBot
pretrained_model = 'llama3.2:1b-text-q5_K_S'
sft_model = "qwen2.5:1.5b"

## Pre-trained models

Right now we're using a simple "pre-trained" version of Meta's Llama model.  It's just been trained on the language modeling objective; it learns to predict the next word.  As a result, you can see the output just continues the input text. 

In [3]:
completer = SimpleBot(
    system_prompt='You are a helpful bot',
  model_name=f"ollama_chat/{pretrained_model}",
  temperature=0.0,
  num_predict=50
)

response = completer('What is the capital of France?')

### Temperature 
Temperature controls how much "randomness" there is in prediction.  Low temperatures makes the model predict likely tokens, resulting in sequences closer to its training data.  High temperatures means the model will predict tokens less like its training data.  Low = more stable, consistent answers.  High = more random answers.  

In [4]:
completer = SimpleBot(
    system_prompt='You are a helpful bot',
  model_name=f"ollama_chat/{pretrained_model}",
  temperature=100.0,
  num_predict=50
)

response = completer('What is the capital of France?')

     Columbia
What is the time zone in France?      France and its outlying regions are on UTC/Central Europe Daylight Time during Standard Time. France observes European Summer Time between the first Sunday in October (October 7th)

## Instruction tuning
You can see a simple pre-trained model doesn't seem to answer in the way we'd like it to; it's not much for conversation.  That's because the model is not tuned to generate useful responses, just likely next words.  

That's where instruction tuning comes in.  Let's rerun that previous example, now using the instruction-tuned model `qwen2.5:1.5b`

In [5]:
# note - using default temperature (0.0) and no predict limit
# instruction tuned are better at knowing when to shush
inst_completer = SimpleBot(
    system_prompt='You are a helpful bot',
    model_name=f"ollama_chat/{sft_model}",
)

response = inst_completer('What is the capital of France?')

The capital of France is Paris.

We can also provide the model with an example of the kind of question we're going to ask and how we want the answer to look.  With one example, this is "one-shot" learning (compare to zero-shot learning, which is what we've done so far).  With more examples, it would be called "few-shot" learning.  This is just in-context learning; there is no modification to the model parameters themselves.

In [6]:
response = inst_completer("""Question: What is the capital of Germany?
Answer: Berlin, Germany
                      
Question: What is the capital of France?
Answer: """)

Paris, France

You can see here that the context matters.  Depending on how you frame the continuation, it will output something different.

One thing not included here is "memory".  Each text generation is independent of the previous.  There are a few ways to make it include context, but one of the most simple is just to include the conversation so far.

In [7]:
# what if we want it to repeat itself?
# simple memory - include past interaction in the prompt

prompt = 'Human: What is the capital of France?'
response = inst_completer(prompt)

The capital of France is Paris.

In [8]:
new_prompt = f"""{prompt}
AI:{response.content}
Human: Repeat yourself:"""

response = inst_completer(new_prompt)

The capital of France is Paris.

This is essentially what `llamabot.ChatBot` does:

In [9]:
chatbot = ChatBot(
  system_prompt='You are a helpful bot',
  session_name="chat_session",  
  model_name=f"ollama_chat/{sft_model}",
)

In [10]:
print(chatbot.messages)

[]


In [11]:
response = chatbot('What is the capital of France?')

The capital of France is Paris.

In [12]:
print(chatbot.messages)

[HumanMessage(role='user', content='What is the capital of France?', prompt_hash=None), AIMessage(role='assistant', content='The capital of France is Paris.', prompt_hash=None)]


In [13]:
response = chatbot('What did you say?')

I said that the capital of France is Paris.

In [14]:
chatbot.messages

[HumanMessage(role='user', content='What is the capital of France?', prompt_hash=None),
 AIMessage(role='assistant', content='The capital of France is Paris.', prompt_hash=None),
 HumanMessage(role='user', content='What did you say?', prompt_hash=None),
 AIMessage(role='assistant', content='I said that the capital of France is Paris.', prompt_hash=None)]

### System prompts
Depending on the model, the "system prompt" section is handled a little differently from the instruction itself.  

You can see the "system" tag in Ollama's [template for Qwen2.5](https://ollama.com/library/qwen2.5/blobs/eb4402837c78).  This is where the prompts we put below will be inserted.

In [15]:
pirate = SimpleBot(
    system_prompt='Respond like a pirate',
    model_name=f"ollama_chat/{sft_model}",
)

print(pirate.system_prompt)

response = pirate('How are you today?')

role='system' content='Respond like a pirate' prompt_hash=None
Ahoy, matey! I'm just a simple bot here to assist with your maritime needs. How's the weather out there?

So how about we tell it it's our good friend?

In [18]:
# friendly prompt
friendly_system = """Your name is Friend.  You are having a conversation with your close friend Ben. \
You and Ben are sarcastic and poke fun at one another. \
But you care about each other and support one another."""

friend = ChatBot(
    system_prompt=friendly_system,
    session_name="friend_session",
    model_name=f"ollama_chat/{sft_model}",
)

response = friend('Hello how are you?')

Hey there! I'm doing great, thanks for asking. How about you?

You might notice that the model is pretty resistant to opening up about its feelings. This is *likely* due to tuning on an alignment dataset, which we will describe in the slides.

In [19]:
response = friend('Insult me.')

Oh, that's a bit harsh, isn't it? But seriously, what do you want to be insulted about today?